In [1]:
import torch

import numpy as np
import torch.nn as nn

from adamp import AdamP
from torch.optim import Adam
from box import Box
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
from architecture.nets.pignn import FlowPIGNN

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# Load the data into train and test datasets
batch_size = 64
data_folder = 'D:\AI2P\data\Case_01\graphs'
data_range = range(30005, 42000 + 1, 5)

class GraphDataset(Dataset):
    def __init__(self, root, data_range, transform=None, pre_transform=None):
        super(GraphDataset, self).__init__(root, transform, pre_transform)
        self.data_range = data_range
        self.graph_paths = self.load_graph_paths()

    def load_graph_paths(self):
        graph_paths = [f"{data_folder}/graph_{i}.pt" for i in self.data_range]
        return graph_paths

    def len(self):
        return len(self.graph_paths)

    def get(self, idx):
        graph_path = self.graph_paths[idx]
        graph_data = torch.load(graph_path)
        return graph_data

dataset = GraphDataset(root=data_folder, data_range=data_range)

total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [4]:
model_cfg = Box({
        'edge_in_dim': 2,
        'node_in_dim': 3,
        'global_in_dim': 1,
        'n_pign_layers': 3,
        'edge_hidden_dim': 50,
        'node_hidden_dim': 50,
        'global_hidden_dim': 50,
        'residual': True,
        'input_norm': True,
        'pign_mlp_params': {'num_neurons': [256, 128],
                           'hidden_act': 'ReLU',
                           'out_act': 'ReLU'},
        'reg_mlp_params': {'num_neurons': [64, 32, 16],
                           'hidden_act': 'ReLU',
                           'out_act': 'ReLU'},
})

In [5]:
num_epochs = 200

def train():
    model = FlowPIGNN(**model_cfg).to(device)
    print(model)

    if torch.cuda.is_available():
        optimizer = AdamP(model.parameters(), lr=1e-3)
    else:
        optimizer = Adam(model.parameters(), lr=1e-3)

    criterion = nn.MSELoss()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=50)

    for epoch in range(num_epochs):
        train_losses = []
        for batch in train_loader:
            node_feat = torch.cat((batch.x.to(device), batch.pos.to(device)), dim=-1)
            edge_feat = batch.edge_attr.to(device)
            glob_feat = batch.global_feats.to(device)
            edge_idx = batch.edge_index.to(device)

            pred = model(node_feat, edge_feat, glob_feat, edge_idx)
            loss = criterion(pred, batch.y)
            train_losses.append(loss)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

        with torch.no_grad():
            model.eval()
            val_losses = []
            for batch in val_loader:
                node_feat = torch.cat((batch.x.to(device), batch.pos.to(device)), dim=-1)
                edge_feat = batch.edge_attr.to(device)
                glob_feat = batch.global_feats.to(device)
                edge_idx = batch.edge_index.to(device)

                pred = model(node_feat, edge_feat, glob_feat, edge_idx)
                val_loss = criterion(pred, batch.y)
                val_losses.append(val_loss)
                model.train()

        learning_rate = optimizer.param_groups[0]['lr']
        print(f"step {epoch}/{num_epochs}, lr: {learning_rate}, training loss: {np.mean(train_losses)}, validation loss: {np.mean(val_losses)}")

        # save model pointer
        torch.save(model.state_dict(), f"pignn_{epoch}.pt")

In [ ]:
train()

FlowPIGNN(
  (gn_layers): ModuleList(
    (0-3): 4 x PIGN()
  )
  (reg): MLP 
  Dimensions : [150, 64, 32, 16, 90000] 
  Hidden Act. : ReLU() 
  Out Act. : ReLU() 
  
)


C:\Users\Matth\AppData\Local\Temp\ipykernel_11640\914739.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  graph_data = torch.load(graph_path)
